<div align="center">
    <h2> <b>DESAFÍO</b></h2>
    <font size = "5">
    <h1> <b>PROYECTO DE CLASIFICACIÓN DE TEXTO</b></h1><br>
</div>
<div>
    <h3> <b>Algoritmos: Clasificación usando Máquinas de Soporte Vectorial con 3000 registros.</b></h3>
</div>

<table>
  <tr><td>
    <img src="emotion.jpg"
         alt="emociones de intensamente en el panel de control"  width="650">
  </td></tr>
  <tr><td align="center">
    <b>Figura 1. Emociones queriendo ser transformadas en etiquetas mediante un modelo de Machine Learning. 
  </td></tr>
</table>

# I° Parte: Preparación de los datos y creación del modelo de machine learning.

## 1) Importar librerías y dataset.

In [1]:
# Importe de librerías de data análisis.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
# Importe de librerías de preprocesado y de machine learning.
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm

In [3]:
# Lectura del archivo train.csv.
dataset = pd.read_csv('text_classification_train.csv')
pd.set_option('display.max_columns',100) # ajuste para ver todas las columnas.
pd.set_option('display.max_rows',10) # ajuste para ver todas las columnas.

In [4]:
#dataset = dataset.drop(dataset.index[3000:]) # eliminé todos los registros a excepción de los 48 primeros.
dataset

,text,emotion,id
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj
...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,edsb738
43406,Always thought that was funny but is it a refe...,6,ee7fdou
43407,What are you talking about? Anything bad that ...,3,efgbhks
43408,"More like a baptism, with sexy results!",13,ed1naf8


In [5]:
# Frecuencia de las emociones predominantes.
dataset['emotion'].value_counts()

27           12823
0             2710
4             1873
15            1857
1             1652
             ...  
6,15,22          1
9,10,19          1
7,10,25          1
7,9,24,25        1
0,1,18           1
Name: emotion, Length: 711, dtype: int64

## 2) Preprocesado del texto (eliminación del ruido en el texto).

In [6]:
import re # Importación de expresiones regulares para eliminar todo caracter distinto a letras.
import nltk # importe de un kit con las principales palabras "inútiles" para el modelo.
nltk.download('stopwords') # descarga de todas las palabras inservibles para el modelo.
from nltk.corpus import stopwords # incorpora la descarga de palabras para utilizarlas.
from nltk.stem.porter import PorterStemmer # permite transformar palabras a su mínima conjugación.

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Danko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Crearemos un corpus, una lista con las 43410 reseñas de texto, pero sin palabras "ruido" (conjunciones, artículos, etc).
corpus = []

In [8]:
# bucle de limpieza (rocorrerá cada dato de la columna "text").
for i in range (0,43410):
    review = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
    # Se crea la variable "review" que se transformará en un texto sin ruido para guardar en el corpus.
    # Función sub: el primer parámetro indica que borraremos todo menos minúsculas y mayúsculas.
    # El segundo parámetro indica qué carácter sustituiremos en el lugar de los carácteres borrados.  
    # El tercer parámetro indica a qué elemento se le hará esta sustitución.
    review = review.lower() # todo a minúsculas.
    review = review.split() # de cadena de strings a lista de palabras.
    ps = PorterStemmer() # objeto para transformar palabras a su mínima expresión (palabras sin conjugar).
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    # Este es un bucle que pasa por todas las palabras de la cadena "i" y mantiene la palabra si ésta
    # no se encuentra en la lista de palabras inservibles que descargamos (las stopwords).
    # Un detalle importante, dado que "stopwords.words('english')" es una lista, si queremos un
    # código más rápido, rodearemos con la función "set()" para que la selección se transforme en un
    # conjunto, y en vez de recorrer todos los elementos hasta encontrar la palabra como lo haría la lista,
    # el conjunto hallará todas las palabras diferentes. Esto desarrollará un algoritmo mucho más rápido que pasará muchas
    # menos veces por la comprobación.
    # nota: Ps.stem no guarda la palabra tal y como se encontraba, sino su mínima conjugación.
    review = ' '.join(review) # transformación de la lista ya filtrada, a cadena de texto.
    corpus.append(review) # añade a la lista corpus cada variable review.

## 3) Bag of Words (bolsa de palabras listas para el modelo).

In [9]:
from sklearn.feature_extraction.text import CountVectorizer # transformación de cadena de palabras a vectores
# de frecuencia. En la documentación se pueden observar muchas funciones, inclusive la automatización
# de la limpieza que realicé anteriormente de forma manual.
cv = CountVectorizer(max_features= 1500) # creamos el objeto CountVectoraizer.
X = cv.fit_transform(corpus).toarray() # hacemos fit_transform y matriz dispersa. Filas son valoraciones, columnas son 0 y 1.
# Es recomendable transformar a una matriz toarray, ya que se crearán muchas columnas.
y = dataset.iloc[:,1] # ya que tenemos las X, obtenemos las y.

## 4) Transformación de la variable objetivo para el MultiLabelBinarizer.

In [10]:
# Función lambda con método split para cambiar los datos de string a lista (se usa la coma para separar cada etiqueta). 
dataset['emotion'] = dataset['emotion'].apply(lambda x: x.split(","))

In [11]:
# Revisamos
dataset['emotion'][20]

['6', '9', '27']

In [12]:
dataset.head(10)

,text,emotion,id
0,My favourite food is anything I didn't have to...,[27],eebbqej
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj
3,To make her feel threatened,[14],ed7ypvh
4,Dirty Southern Wankers,[3],ed0bdzj
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,[26],edvnz26
6,Yes I heard abt the f bombs! That has to be wh...,[15],ee3b6wu
7,We need more boards and to create a bit more s...,"[8, 20]",ef4qmod
8,Damn youtube and outrage drama is super lucrat...,[0],ed8wbdn
9,It might be linked to the trust factor of your...,[27],eczgv1o


## 5) Multi-label binarizer (matriz dispersa para las variables objetivos).

In [13]:
multilabel = MultiLabelBinarizer() # creación del objeto para crear la matriz dispersa a partir del texto 
y = multilabel.fit_transform(dataset['emotion']) # ajuste y transformación del modelo
# Revisamos
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
multilabel.classes_ # método para obtener las etiquetas de cada emoción.

array(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '3',
       '4', '5', '6', '7', '8', '9'], dtype=object)

In [15]:
pd.DataFrame(y, columns = multilabel.classes_).head(10) # dataframe para ver la matriz con sus respectivas etiquetas.

,0,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [16]:
X.shape , y.shape # shape para ver la cantidad de filas y columnas de cada matriz de dispersión.

((43410, 1500), (43410, 28))

### Ya están listas las variables predictoras y las variables objetivo para comenzar la fase de entrenamiento.

## 6) Fase de entrenamiento.

In [17]:
# Separación de las variables en variables de entrenamiento (80%) y variables de testeo (20%).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [18]:
y_train.shape # shape para ver la cantidad de datos de entrenamiento.

(34728, 28)

## 7) Modelo SVC (máquina de vectores de soporte C).

In [ ]:
from sklearn.multiclass import OneVsRestClassifier # Importe de algoritmo de clasificación multiclase.
from sklearn.svm import SVC # importe del modelo de clasificación de vectores de soporte C.

clf = OneVsRestClassifier(SVC(kernel='linear'))
# Creamos el objeto OneVsRestClassifier y le asignamos como parámetro el modelo SVC. Este estimador utiliza el método de
# relevancia binaria para realizar las clasificaciones multietiqueta, entrenando un clasificador binario independiente para
# cada etiqueta.
clf.fit(X_train, y_train) # ajuste de estimadores subyacentes (datos de entrenamiento X e y).

y_pred = clf.predict(X_test) # predicción de objetivos de varias etiquetas usando el 20% de los datos X de testing.

In [ ]:
#pd.DataFrame(y_pred, columns = multilabel.classes_).head(10) # dataframe de y_pred con las respectivas etiquetas. 

## 8) Evaluación del modelo.

In [ ]:
# importe de librerías para medir el accuracy, la matriz de confusión y otras métricas de precisión.
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# El método score devuelve la precisión media en los datos de prueba y las etiquetas dadas.
clf.score(X_test, y_test)

In [ ]:
# Tasa de acierto
accuracy_score(y_test,y_pred)

In [ ]:
# Importe de matriz de confusión multietiqueta
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(y_test, y_pred) # imprime matrices de confusión para las predicciones de todas las etiquetas. 

In [ ]:
# Imprimiremos la precisión, el recall y el F1-score para las predicciones de cada una de las etiquetas.
# Pero antes, cambiaremos el nombre de las etiquetas (nombre de la emoción). Crearemos una lista llamada
# label_names, y la entregaremos como parámetro a la función classification_report. El orden no es el presentado en el archivo
# emotion.txt, si no el que entregó el MultiLabelBinarizer.
print(classification_report(y_test, y_pred))

### Evaluación y Conclusión..

### 1.- ACCURACY: La tasa de acierto de la función "accuracy_score" es de un 36%, sin embargo, se utiliza una matriz de confusión multietiqueta para identificar cuáles son las emociones con más alto accuracy. 

### 2.- PRECISIÓN: La calidad del etiquetado que predijo el modelo SVC, fue bastante bueno. Al menos 24 de las 28 emociones, fueron predichas con una precisión igual o superior 50%.

### 3.- RECALL: La cantidad de etiquetas positivas que el modelo logró identificar correctamente, en general, fue muy bajo. El modelo solo logró identificar 5 etiquetas con un porcentaje igual o superior al 50%. El mejor recall fue el de la etiqueta #15, correspondiente a la emoción de gratitud.

### 4.- F1-SCORE: Utilizando un margen superior al 50%, las etiquetas con mejor F1_score (media armónica entre precisión y recall) son 8 emociones.

### CONCLUSIÓN.

### El modelo SVC para clasificación multietiqueta, presenta un nivel de ACCURACY bajísimo, por lo que se observa con minuciosidad, la PRECISIÓN Y EL RECALL de los modelos. 

### Como resultado, podemos ver una PRECISIÓN muy alta del etiquetado de cada emoción, como por ejemplo en la "gratitud" con un 93% de precisión y en la "curiosidad", con un 100% de aciertos. Sin embargo, se observa un RECALL bajísimo en comparación con los valores obtenidos en la métrica anterior, y claro, al ver las matrices de confusión, se puede observar que en la emoción "curiosidad", por ejemplo, predice 18 de las 18 predicciones realizadas, sin embargo, le faltaron 417 comentarios que etiquetar con esta emoción.

### Se realizan dos conclusiones:

### 1) El modelo fue bastante tímido para realizar predicciones. Podríamos decir que el modelo se atrevió a predecir solo cuando estaba muy seguro de acertar. Esto se evidencia por el número de comentarios no etiquetados (ESCRIBIR EL NÚMERO AQUÍ), lo que se resume en un (ESCRIBIR EL NÚMERO AQUÍ)% de comentarios con su contenido desconocido.

### 2) El etiquetado es, en gran medida muy preciso, sin embargo, en algunas etiquetas es altamente confiable y en otras lo es nulamente, por lo que hay que es imprecindible realizar una lista de las emociones más confiables, si lo que se desea es realizar KPIs fidedignas. 

# II° Parte: Clasificación final del dataset "text_classification_test" (predicción).

## 1) Lectura de los datos.

In [ ]:
dataset_text_classification = pd.read_csv('text_classification_test.csv')

In [ ]:
dataset_text_classification

## 2) Preprocesado de las reseñas del csv de testing (limpieza).

In [ ]:
# corpus para la predicción
corpus_text_classification = []

In [ ]:
# bucle de limpieza
for i in range (0,5427):
    review_text_classification = re.sub('[^a-zA-Z]', ' ', dataset_text_classification['text'][i])
    review_text_classification = review_text_classification.lower()
    review_text_classification = review_text_classification.split()
    ps_text_classification = PorterStemmer()
    review_text_classification = [ps_text_classification.stem(word) for word in review_text_classification if not word in set(stopwords.words('english'))]
    review_text_classification = ' '.join(review_text_classification)
    corpus_text_classification.append(review_text_classification)

## 3) Bag of Words.

In [ ]:
cv_text_classification = CountVectorizer(max_features= 1500)
X_text_classification = cv_text_classification.fit_transform(corpus_text_classification).toarray()

In [ ]:
X_text_classification

## 4) Nube de puntos.

In [ ]:
from wordcloud import WordCloud # importamos librería

In [ ]:
# Creamos el objeto. Fondo negro, alto y ancho.
wc_text_classification = WordCloud(background_color='black', height=600, width=400)

In [ ]:
# WordCloud solo acepta texto, así que se transforma la lista de strings (corpus) en un string.
corpus_wc = ''
for i in corpus_text_classification:
    corpus_wc = corpus_wc + i

In [ ]:
# Se genera el wordcloud entregándo como argumento el corpus_wc 
wc_text_classification.generate(corpus_wc)

In [ ]:
# Se genera un archivo png para insertar una imagen en la próxima celda. Queda muy bonita.
wc_text_classification.to_file('wordcloud_text_classification.png')

<div align="center">
    <img src="wordcloud_text_classification.png" alt="Nube de Palabras"  width="300">
</div>

 ## 5) Predicción usando nuestro modelo de máquinas de soporte vectorial.   

In [ ]:
y_pred_text_classification = clf.predict(X_text_classification) # realizamos la predicción

In [ ]:
pd.DataFrame(y_pred_text_classification, columns = multilabel.classes_)

## 6) Transformación Inversa (transformar matriz dispersa a columna de etiquetas)

In [ ]:
# Utilizamos el método "inverse_transform" de la librería MultiLabelBinarizer para volver a tener el nombre de nuestras
# etiquetas (números del 0 al 27).
y_pred_transformado = multilabel.inverse_transform(y_pred_text_classification) 

## 7) Agregar las predicciones como columna al dataset

In [ ]:
dataset_text_classification['Emoción'] = y_pred_transformado

In [ ]:
dataset_text_classification

## 8) Transformación de la columna Emoción a str con la función lambda y el método join.

In [ ]:
dataset_text_classification['Emoción'] = dataset_text_classification['Emoción'].apply(lambda x: ','.join(x))

In [ ]:
dataset_text_classification['Emoción'][0]

In [ ]:
type(dataset_text_classification['Emoción'][0])

# III° Parte: KPIs.

## KPI
### En base a la clasificación de los comentarios, realizaremos un KPI capaz de medir el nivel de satisfacción global (explicar significado y justificar selección). 

In [ ]:
# Para realizar un KPI valioso, veremos el nivel de comentarios positivos, negativos y neutros,
# que hay en nuestro dataset.
dataset_text_classification['Emoción'].value_counts()

### Observemos que hay 1895 datos sin clasificar, lo que significa que en este caso, el modelo etiquetó el 65% de los comentarios.

In [ ]:
pd.set_option('display.max_rows',10) # ajuste para ver todas las columnas.
dataset_text_classification['Emoción'].value_counts()

# IV° Parte: Post-desarrollo (zona de preguntas y respuestas). 

### 1.	¿De qué manera se puede complementar la solución? Pensar en propuestas para el cliente.
#### Una forma de complementar la solución, sería aumentando el valor del modelo de clasificación de texto. Esto quiere decir, alojar los modelos de machine learning en un sistema web en la nube, que, a pesar de requerir un esfuerzo relativo mayor, entregaría muchas más funcionalidades. Por ejemplo, la posibilidad de permitir al usuario elegir el modelo a utilizar, conociendo a priori el tiempo de ejecución, certeza y el porcentaje de etiquetado que entrega cada modelo, la posibilidad de realizar una cascada de los datos a través de los modelos, comparar entre los resultados de los modelos o acceder a funcionalidades de visualización de resultados.

### 2.	¿Cómo se podría simplificar la tarea?
#### El modelo podría simplificarse eludiendo el preprocesado de texto, etapa que requiere de mucho tiempo si el proceso se realiza con grandes cantidades de datos. Sin embargo, la cantidad de tiempo que demora el entrenamiento, sería mayor, y probablemente, los resultados serían menos exactos. Otra forma podría ser no utilizar multietiquetado, para no confundir al modelo (solo trabajar con clases), o bien, reduciendo la cantidad de etiquetas, lo que por un lado permitiría mejores niveles de asertividad, lo que lamentablemente haría perder el nivel de detalle de emociones que presenta el dataset.


### 3.	¿Cuáles pueden ser las limitaciones, riesgos, sesgos de los modelos al implementar este tipo de soluciones? 
#### Las limitaciones de estos modelos pasan por ser soluciones generalmente costosas y que pueden llevar mucho tiempo dada la binarización de los datos tipo texto y el multietiquetado. En este caso en particular, además, el multietiquetado contenía 28 valores distintos, por lo que la cantidad de veces que el código es aplicado, aumenta. Por otra parte, estos modelos presentan dos situaciones que aumentan el riesgo de su uso; 1) Los modelos no clasifican a todos los comentarios, por ende, se estarían tomando decisiones sin la totalidad de los datos. 2) El sesgo que puede presentar este modelo,  ya que pueden existir errores en los datos, cosa que se infiere en el enunciado del desafío: “La empresa ya ha clasificado manualmente alrededor de 48 comentarios, con 28 emociones diferentes”, sin embargo, un poco más arriba, la empresa evidencia que “Actualmente, los comentarios se clasifican manualmente, lo que es un proceso lento y propenso a errores”, lo que podría conllevar a que el resultado final, necesariamente presente sesgo (error en la captura de datos). Por otra parte, también puede ser posible que estos datos presenten sesgo algorítmico, sesgo que afecta principalmente a las menorías o a aquellos grupos de datos que no están bien representados. Los modelos pueden basarse en gran medida en etiquetas que están correlacionados con otras etiquetas y asignar un mal etiquetado a un comentario.


### 4.	¿Qué otras cosas hay que considerar al momento de implementar un proyecto como este?
#### Primordial en primer lugar es realizar una buena preparación de los datos para que no tengamos que descartar el modelo por malos resultados. Es importante eliminar ciertos caracteres y palabras clave (stopwords). Además, es importante utilizar diferentes funciones para algunos procesos, como por ejemplo CountVectorizer vs TfidfVectorizer, para elegir la que mejor se ajusta a los datos de entrenamiento (en este caso, TfidfVectorizer emplea ponderación inversa, es decir, el modelo otorga menor ponderación a las palabras con menor frecuencia, aprendiendo que estas tienen menor importancia). Por otra parte, también es necesario practicar con diferentes modelos de machine learning y jugar con sus parámetros, para ver qué algoritmo se ajusta mejor a los requerimientos del cliente (mayor velocidad, mayor precisión o mayor exhaustividad).


# V° Parte: Entregable (exportar dataset con columna de predicciones).

In [ ]:
dataset_text_classification.to_csv('text_classification_entrega.csv', index=False)